<a href="https://colab.research.google.com/github/gokulchand98/PyForML/blob/main/python_refresher_student_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Refresher Lab — FAANG-Level Hands-On

**Goal:** Refresh Python fundamentals + interview patterns the way FAANG expects ML engineers to code.

**Outcome:** You can write clean, efficient Python, explain complexity, and avoid classic interview gotchas.

---

## How to Start
1. Open this notebook in Colab
2. Complete all **TODO** sections
3. Restart runtime → Run all

---

## Lab Rules (FAANG Style)
- ✅ Complexity for each core function
- ✅ Handle edge cases
- ✅ Prefer O(n) over O(n²) when feasible
- ✅ Add tests
- ❌ No hardcoding outputs


In [11]:
from __future__ import annotations
from collections import Counter, defaultdict, deque
import heapq

# NOTE: Use this helper for quick sanity checks
def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

## Section 1 — Python Idioms & Core Data Structures

### Task 1.1: Comprehensions (no unnecessary loops)

Create (without loops):
- `even_squares`: squares of even numbers from 1..20
- `pairs`: list of (x, x*x) for x=1..10
- `flattened`: flatten `nested`

**Explain:** Why can comprehensions be faster than manual loops?

In [12]:
# TODO:
# Create even_squares
# HINT: range(1, 21), filter even with x % 2 == 0
even_squares = [x * x for  x in range(1 , 21) if x % 2 == 0]

# TODO:
# Create pairs
pairs = [(x,x*x) for  x in range(1 , 11)]

# TODO:
# Flatten nested
nested = [[1, 2], [3, 4], [5, 6]]
# HINT: nested loops inside a single comprehension
flattened = [element for row in nested for element in row]

check('even_squares', even_squares == [4, 16, 36, 64, 100, 144, 196, 256, 324, 400])
check('pairs', pairs[:3] == [(1, 1), (2, 4), (3, 9)] and pairs[-1] == (10, 100))
check('flattened', flattened == [1, 2, 3, 4, 5, 6])

OK: even_squares
OK: pairs
OK: flattened


### Task 1.2: Counter (frequency counting)

Given `s`, compute:
- character frequency using `Counter`
- top 2 most common characters

**Interview angle:** When is Counter preferable to sorting?

In [13]:
s = 'mississippi'


freq = Counter(s)
top2 = freq.most_common(2)

print(freq)
print(top2)
check('top2', top2[0][0] == 'i' and top2[0][1] == 4)

Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
[('i', 4), ('s', 4)]
OK: top2


### Task 1.3: defaultdict (grouping)

Group words by first letter.

**HINT:** `defaultdict(list)`

**Gotcha:** Don’t use a normal dict unless you handle missing keys.

In [14]:
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry', 'avocado']


groups = defaultdict(list)
for w in words:
  groups[w[0]].append(w)


print(dict(groups))
check('groups[a]', dict(groups)['a'] == ['apple', 'apricot', 'avocado'])

{'a': ['apple', 'apricot', 'avocado'], 'b': ['banana', 'blueberry'], 'c': ['cherry']}
OK: groups[a]


## Section 2 — FAANG Core Patterns

### Task 2.1: Two Sum (Hash Map Pattern)

**FAANG Classic.** Return indices of two numbers that sum to target.

# HINT:
- Use a dict: value -> index
- For each `x`, check if `target-x` has been seen

**Explain:** Why is this O(n) time?

In [15]:
def two_sum(nums, target):

  hashMap = {}
  for index , value in enumerate(nums):
    diff = target -value
    if diff in hashMap:
      return [hashMap[diff], index]
    hashMap[value] = index
  return []

 #answer: O(n) is beacuse of The loop runs once for each element in nums

check('two_sum_1', two_sum([2, 7, 11, 15], 9) == [0, 1])
check('two_sum_2', two_sum([3, 2, 4], 6) == [1, 2])
check('two_sum_dup', two_sum([3, 3], 6) == [0, 1])

OK: two_sum_1
OK: two_sum_2
OK: two_sum_dup


### Task 2.2: Longest Substring Without Repeating Characters (Sliding Window)

**FAANG Classic/Gotcha:** naive O(n²) solutions time out.

# HINT:
- Keep a map char -> most recent index
- Maintain a window `[l..r]` with unique chars

**Explain:** What invariant does the window maintain?

In [16]:
def length_of_longest_substring(s: str) -> int:
    hashMap = {}
    start = 0
    max_len = 0
    for i , ch in enumerate(s):
      if ch in hashMap:
        start = max(start, hashMap.get(ch))
      max_len = max(max_len , (i-start+1))
      hashMap[ch] = i +1
    return max_len
#answer:The invariant isthe hashMap[ch] stores the index immediately after the last occurrence of ch,when a repeated character ch is seen at index i, then updating the start by start = max(start, hashMap[ch]) which removes the previou occurence of ch from the window and prevents start from moving backward, so the window always represents the largest valid substring ending at index i.

check('ls_abcabcbb', length_of_longest_substring('abcabcbb') == 3)
check('ls_bbbbb', length_of_longest_substring('bbbbb') == 1)
check('ls_pwwkew', length_of_longest_substring('pwwkew') == 3)
check('ls_empty', length_of_longest_substring('') == 0)

OK: ls_abcabcbb
OK: ls_bbbbb
OK: ls_pwwkew
OK: ls_empty


### Task 2.3: Product of Array Except Self (Prefix/Suffix)

**FAANG Classic/Gotcha:**
- No division
- O(n) time

# HINT:
- Build prefix products in one pass
- Multiply by suffix products in reverse pass

**Explain:** Why does this work with zeros?

In [17]:
def product_except_self(nums):
    n = len(nums)
    answer = [1] * n
    rp  = 1
    for i in range(1,n):
      rp = rp * nums[i-1]
      answer[i] = rp
    rp = 1
    for i in range(n-2,-1,-1):
      rp=rp * nums[i + 1]
      answer[i] = answer[i] * rp
    return answer

check('pes_1234', product_except_self([1, 2, 3, 4]) == [24, 12, 8, 6])
check('pes_zero', product_except_self([-1, 1, 0, -3, 3]) == [0, 0, 9, 0, 0])

OK: pes_1234
OK: pes_zero


## Section 3 — FAANG Hard (Optional / Bonus)

### Task 3.1: Sliding Window Maximum (Deque)

**FAANG Classic:** maintain a decreasing deque of indices.

# HINT:
- Pop from left if out of window
- Pop from right while current value is larger

**Explain:** Why is this amortized O(n)?

In [18]:
def max_sliding_window(nums, k):
    # TODO
    ...

check('msw', max_sliding_window([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7])

AssertionError: Failed: msw

### Task 3.2: LRU Cache (Hash Map + Doubly Linked List)

**FAANG Hard/Gotcha:** You must get O(1) `get` and `put`.

# HINT:
- Dict: key -> node
- Doubly linked list: move-to-front on access

You can implement this later if you run out of time today.

---

## Submission Checklist
- All TODOs completed
- All tests passing
- Explain prompts answered
- Complexity noted
